In [1]:
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import StratifiedKFold


In [2]:
train=pd.read_csv('../balanced_cleaned_trian_dataset.csv',index_col=0)
test_set=pd.read_csv('../testset.csv',index_col=0)

train_copy=train.drop('id',axis=1)
X = train_copy.drop('smoking', axis=1)
y = train_copy['smoking']

test_set=test_set.drop('id',axis=1)
X_test_new= test_set.drop('smoking', axis=1)
y_test_new = test_set['smoking']

In [ ]:
import optuna
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score
import numpy as np

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def objective(trial):
    
    n_d = trial.suggest_int("n_d", 16, 64, step=8)        
    n_a = trial.suggest_int("n_a", 16, 64, step=8)        
    n_steps = trial.suggest_int("n_steps", 4, 10)         
    gamma = trial.suggest_float("gamma", 1.0, 2.0)        
    lambda_sparse = trial.suggest_float("lambda_sparse", 1e-6, 1e-4, log=True)  
    lr = trial.suggest_float("lr", 5e-4, 5e-3, log=True)  
    max_epochs = trial.suggest_int("max_epochs", 50, 200) 

    scores = []

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
        y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]   

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train_fold)
        X_val_scaled   = scaler.transform(X_val_fold)

        
        X_train_fold = X_train_scaled.astype(np.float32)
        X_val_fold   = X_val_scaled.astype(np.float32)
        y_train_fold = y_train_fold.values.astype(np.int64)
        y_val_fold   = y_val_fold.values.astype(np.int64)

        model = TabNetClassifier(
            n_d=n_d,
            n_a=n_a,
            n_steps=n_steps,
            gamma=gamma,
            lambda_sparse=lambda_sparse,
            optimizer_fn=torch.optim.Adam,
            optimizer_params=dict(lr=lr),
            verbose=0,
            seed=42
        )

        model.fit(
            X_train_fold, y_train_fold,
            # eval_set=[(X_val_fold, y_val_fold)],
            # eval_metric=['accuracy'],
            max_epochs=max_epochs,
            # patience=15,
            batch_size=1024,
            virtual_batch_size=128
        )

        y_pred_val = model.predict(X_val_fold)
        acc = accuracy_score(y_val_fold, y_pred_val)
        
        scores.append(acc)


    return np.mean(scores)


sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=20)

print("best_params:", study.best_params)
print("best_value:", study.best_value)


[I 2025-08-06 20:50:35,107] A new study created in memory with name: no-name-94eca193-bcd5-4fab-a25d-5aea8e09ad8a


Stop training because you reached max_epochs = 58 with best_epoch = 57 and best_val_0_accuracy = 0.737


/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Stop training because you reached max_epochs = 58 with best_epoch = 57 and best_val_0_accuracy = 0.75875


/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


...


Early stopping occurred at epoch 44 with best_epoch = 29 and best_val_0_accuracy = 0.7625


/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Early stopping occurred at epoch 45 with best_epoch = 30 and best_val_0_accuracy = 0.76125


/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2025-08-06 22:39:04,697] Trial 19 finished with value: 0.7694500000000001 and parameters: {'n_d': 64, 'n_a': 48, 'n_steps': 7, 'gamma': 1.218988225542085, 'lambda_sparse': 2.3795180436386398e-05, 'lr': 0.00294182835877787, 'max_epochs': 157}. Best is trial 11 with value: 0.7777000000000001.


best_params: {'n_d': 64, 'n_a': 48, 'n_steps': 10, 'gamma': 1.0087473669686107, 'lambda_sparse': 2.1458782520594695e-05, 'lr': 0.0046293887561929875, 'max_epochs': 60}
best_value: 0.7777000000000001


In [ ]:

X_train_f, X_val_f, y_train_f, y_val_f = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train_f_scaled = scaler.fit_transform(X_train_f)
X_val_f_scaled   = scaler.transform(X_val_f)
X_test_new_scaled  = scaler.transform(X_test_new)


X_train_scaled_np = X_train_f_scaled.astype(np.float32)
y_train_np = y_train_f.values.astype(np.int64)

X_val_scaled_np = X_val_f_scaled.astype(np.float32)
y_val_np = y_val_f.values.astype(np.int64)

X_test_new_scaled_np = X_test_new_scaled.astype(np.float32)
y_test_new_np = y_test_new.values.astype(np.int64)


tabnet_model = TabNetClassifier(
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=0.0046293887561929875),
    scheduler_params={"step_size":10, "gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    verbose=10,
    n_d=64, n_a=48, n_steps=10, gamma=1.0087473669686107,
    lambda_sparse=2.1458782520594695e-05,
    seed=42
)



tabnet_model.fit(
    X_train=X_train_scaled_np, y_train=y_train_np,
    batch_size=1024,
    virtual_batch_size=128,
    max_epochs=33,
    num_workers=4
)

y_train_pred = tabnet_model.predict(X_train_scaled_np)
train_acc = accuracy_score(y_train_np, y_train_pred)

print(f"TabNet Train Accuracy: {train_acc:.4f}")
# print(classification_report(y_train_np, y_train_pred, digits=4))


y_pred = tabnet_model.predict(X_test_new_scaled_np)
acc = accuracy_score(y_test_new_np, y_pred)

print(f"\n TabNet Test Accuracy: {acc:.4f}")
print(classification_report(y_test_new_np, y_pred, digits=4))

/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 1.14985 |  0:00:26s
epoch 10 | loss: 0.5395  |  0:04:45s
epoch 20 | loss: 0.50737 |  0:09:04s
epoch 30 | loss: 0.48219 |  0:13:23s
TabNet Train Accuracy: 0.7764

 TabNet Test Accuracy: 0.7755
              precision    recall  f1-score   support

           0     0.8555    0.6630    0.7470      1000
           1     0.7249    0.8880    0.7982      1000

    accuracy                         0.7755      2000
   macro avg     0.7902    0.7755    0.7726      2000
weighted avg     0.7902    0.7755    0.7726      2000



In [4]:
tabnet_model.save_model("tabnet_model(2w)")
loaded_model = TabNetClassifier()


loaded_model.load_model("tabnet_model(2w).zip")

Successfully saved model at tabnet_model(2w).zip


/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:454: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
    y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]   

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_fold)
    X_val_scaled   = scaler.transform(X_val_fold)

    
    X_train_fold = X_train_scaled.astype(np.float32)
    X_val_fold   = X_val_scaled.astype(np.float32)
    y_train_fold = y_train_fold.values.astype(np.int64)
    y_val_fold   = y_val_fold.values.astype(np.int64)

    model = TabNetClassifier(
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=0.0046293887561929875),
        scheduler_params={"step_size":10, "gamma":0.9},
        scheduler_fn=torch.optim.lr_scheduler.StepLR,
        verbose=10,
        n_d=64, n_a=48, n_steps=10, gamma=1.0087473669686107,
        lambda_sparse=2.1458782520594695e-05,
        seed=42
    )

    model.fit(
        X_train_fold, y_train_fold,
        batch_size=1024,
        virtual_batch_size=128,
        max_epochs=33,
        num_workers=4
    )

    y_pred_val = model.predict(X_val_fold)
    acc = accuracy_score(y_val_fold, y_pred_val)

    y_train_pred = tabnet_model.predict(X_train_fold)
    train_acc = accuracy_score(y_train_fold, y_train_pred)

    
    print(f"TabNet Train Accuracy: {train_acc:.4f}")


    print(f"\n TabNet Test Accuracy: {acc:.4f}")
    


   


